In [73]:
import time
import json
import numpy as np
import sympy as sp
from itertools import combinations
import chipsplitting.hyperfield.utils as utils
from chipsplitting import PascalForm
from chipsplitting.hyperfield import (
    HyperfieldPascalForm as HVPascal, 
    HyperfieldHomogeneousLinearSystem as HVLinearSystem,
    HyperfieldVector
)
from chipsplitting import pairing_matrix, print_matrix, PascalForm

## Utils

In [277]:
# utils
def save_fundamental_models(model, json_file_name):
    with open(json_file_name, 'w') as json_file:
        json.dump([{"support": supp, "solution": [x for x in list(solution_set)[0]] if len(solution_set) > 0 else [] } for supp, solution_set in model], json_file, default=float)
    
    print(f"JSON dump has been written to {json_file_name}")

def create_symbols(n):
    # Create a string pattern for n symbols w0, w1, ..., wn-1
    pattern = ' '.join(f'w{i}' for i in range(n))
    # Create the symbols using the pattern
    symbols = sp.symbols(pattern)
    return symbols

# compute_coefficients is not used currently
def compute_coefficients(mode, n):
    coeff = {}
    if mode == "1-t":
        for k in range(n + 1):
            coeff[f"t{k}"] = comb(n,k) * (-1)**(k)
        return coeff
    elif mode == "t":
        for k in range(n + 1):
            coeff[f"t{k}"] = 1 if k == n else 0
        return coeff
    else:
        raise ValueError()
        
def create_w_symbols(n):
    # Create a string pattern for n + 1 symbols
    pattern = ' '.join(f'w{i}' for i in range(n))
    # Create the symbols using the pattern
    symbols = sp.symbols(pattern)
    return symbols

def get_linear_equations(support):
    t = sp.symbols('t')
    ws = create_w_symbols(len(support))
    eqn = 0
    for index, coord in enumerate(support):
        col, row = coord
        expression = ws[index] * t**col * (1 - t)**row
        expanded_expression = sp.expand(expression)
        eqn += expanded_expression
    eqn += -1
    return eqn

def max_degree(support):
    max_degree = 0
    for col, row in support:
        if col + row > max_degree:
            max_degree = col + row
    return max_degree

def contains_var(sol):
    for x in sol:
        if not x.is_number:
            return True
    return False
    
def is_valid_solution_set(solutions):
    #if not solutions:
    #    return False
    for sol in solutions:
        if contains_var(sol):
            return False
    return True

def create_augmented_matrix(num_vars, equations):
    ws = create_symbols(num_vars)
    matrix = []
    b = []
    
    for eqn in equations:
        row = []
        for w in ws:
            coeff = eqn.coeff(w)
            row.append(coeff)
        matrix.append(row)
        
        constant = eqn.as_independent(*ws)[0]
        b.append(constant)

    A = np.array(matrix, dtype='float64')
    b = np.array(b, dtype='float64').reshape(len(b), 1)
    augmented = np.concatenate((A, b), axis = 1)
    return augmented

def create_matrix(num_vars, equations):
    ws = create_symbols(num_vars)
    matrix = []
    
    for eqn in equations:
        row = []
        for w in ws:
            coeff = eqn.coeff(w)
            row.append(coeff)
        matrix.append(row)

    A = np.array(matrix, dtype="float")
    return A

def create_b(num_vars, equations):
    ws = create_symbols(num_vars)
    b = []
    
    for eqn in equations:
        constant = eqn.as_independent(*ws)[0]
        b.append(constant)

    b = np.array(b, dtype='float')
    return -b

def is_fundamental(support_as_index):
    t = sp.symbols('t')
    support = [utils.to_coordinate(s) for s in support_as_index]
    ws = create_w_symbols(len(support))
    max_power_t = max_degree(support)
    expression = get_linear_equations(support)
    linear_equations = {}
    
    for power in range(max_power_t + 1):
        coeff = expression.coeff(t, power)
        linear_equations[f"t{power}"] = coeff
    
    system = list(linear_equations.values())

    # if len(system) < len(ws):
    #     return (False, None)

    # augmented = create_augmented_matrix(len(ws), system)
    # matrix = create_matrix(len(ws), system)
    # if np.linalg.matrix_rank(matrix) < len(ws):
    #    return (False, None)

    #b = create_b(len(ws), system)
    #pinv_matrix = np.linalg.pinv(matrix)
    #solution = np.dot(pinv_matrix, b)

    solution_set = sp.linsolve(system, *ws)
    is_valid = is_valid_solution_set(solution_set)
    
    return (is_valid, solution_set if is_valid else None)

**Task**: For $\Delta_2$, find all positive supports of size $3$ of valid hyperfield configurations of degree $d = 1,2,3$.

## Compute possible supports of fundamental models

Iterating through all possible supports is not feasible. Luckily, we can exclude invalid supports using the invertibility criterion.

In [3]:
possible_supports = {}

def find_positive_supports(pos_support_size, mode = "fast", expand = False):
    degree_end = 2 * pos_support_size - 3
    # hyperfield variety generated by all pascal equations
    S = {}
    for d in range(1, degree_end + 1):
        num_cells = utils.gauss(d+1)
        # positive cells are all cells except the (0,0) cell
        num_pos_cells = num_cells - 1
        if num_pos_cells < pos_support_size:
            continue
            
        base_types = ["diag", "row", "col"]
        A = [HVPascal(d, b, k) for b in base_types for k in range(d + 1)]
        linear_system = HVLinearSystem(A)
        
        # I believe quick_solve_loop has a bug
        if mode == "fast":
            solutions = linear_system.quick_solve_loop(pos_support_size)
        else:
            solutions = linear_system.quick_solve(pos_support_size)
        
        if expand:
            def expand_solution(sol):
                num_selections = pos_support_size - len(sol)
                assert num_selections > 0
                domain = [x for x in range(utils.gauss(d + 1)) if x not in sol]
                combi_from_domain = list(combinations(domain, num_selections))
                return [tuple(sorted(sol + c)) for c in combi_from_domain]

            expanded_solutions = [sol for sol in solutions if len(sol) == pos_support_size]
            for sol in [sol for sol in solutions if len(sol) < pos_support_size]:
                expanded_solutions += expand_solution(sol)
            solutions = list(set(expanded_solutions))
        
        S[d] = solutions
        print(f"d={d}, #solutions={len(solutions)}")
    return S

def apply_symmetry(config):
    def swap(tuple):
        return (tuple[1], tuple[0])
    def to_array_index(tuple):
        return utils.get_array_index(tuple[0], tuple[1])
    return tuple(sorted([to_array_index(swap(utils.to_coordinate(index))) for index in config]))

In [85]:
n_begin = 6
n_end = 6
for pos_support_size in range(n_begin + 1, n_end + 2):
    print()
    print(f"n={pos_support_size - 1}")
    possible_supports[pos_support_size] = find_positive_supports(pos_support_size, mode = "fast", expand = True)

print("Reducing possible solutions by applying symmetry...")

possible_supports_copy = possible_supports.copy()

for n, d_to_solutions in possible_supports.items():
    d_to_solutions_copy = {}
    for d, solutions in d_to_solutions.items():
        solutions_copy = set()
        for config in solutions:
            if apply_symmetry(config) not in solutions_copy:
                solutions_copy.add(config)
        d_to_solutions_copy[d] = list(solutions_copy)
    possible_supports_copy[n] = d_to_solutions_copy

possible_supports = possible_supports_copy

for n, d_to_solutions in possible_supports.items():
    print("")
    print(f"n={n-1}")
    for d, solutions in d_to_solutions.items():
        print(f"d={d}, #solutions={len(solutions)}")


n=6
d=3, #solutions=89
d=4, #solutions=2431
d=5, #solutions=21136
d=6, #solutions=104439
d=7, #solutions=360471
d=8, #solutions=982891
d=9, #solutions=2319165
d=10, #solutions=4854071
d=11, #solutions=9418679
Reducing possible solutions by applying symmetry...

n=6
d=3, #solutions=48
d=4, #solutions=1226
d=5, #solutions=10635
d=6, #solutions=52284
d=7, #solutions=180527
d=8, #solutions=491601
d=9, #solutions=1160287
d=10, #solutions=2427353
d=11, #solutions=4710754


In [292]:
json_file_name = "section8_n6_possible_supports.json"
with open(json_file_name, 'w') as json_file:
    json.dump(possible_supports, json_file, default=int)

print(f"JSON dump has been written to {json_file_name}")


JSON dump has been written to section8_n6_possible_supports.json


In [64]:
def check_solution_set(solution_set):
    if len(solution_set) != 1:
        return False
    
    for solution in solution_set:
        for x in solution:
            if x <= 0:
                return False

    return True

def filter_fundamental_models(fundamental_models):
    return [(support, solution_set) for support, solution_set in fundamental_models if check_solution_set(solution_set)]

def find_fundamental_models(n, degrees):
    if isinstance(degrees, int):
        degrees = [degrees]
        
    fundamental_models = []
    
    for d in degrees:
        start = time.time()
        supports_to_check = possible_supports[n + 1][d]

        print("")
        print(f"d={d}: {len(supports_to_check)} supports to check")

        for index, support in enumerate(supports_to_check):
            if index == 100:
                elapsed = time.time() - start
                print(f"Processed 100 items. Elapsed time: {round(elapsed, 2)}s. Estimate: {round(len(supports_to_check) / 100 * elapsed / 60, 2)}min")
                
            is_valid, solution = is_fundamental(support) 
            if is_valid:
                fundamental_models.append((support, solution))

        end = time.time()
        print(f"Processed all items. Elapsed time: {round(end - start, 2)}s")

    return fundamental_models

## Run computations for $n=6$

### $d=3,4,5$

In [270]:
fundamental_models = find_fundamental_models(6, [3,4,5])
print()
print(f"Found {len(fundamental_models)} potential fundamental models")


d=3: 48 supports to check
Processed all items. Elapsed time: 0.07s

d=4: 1226 supports to check
Processed 100 items. Elapsed time: 0.09s. Estimate: 0.02min
Processed all items. Elapsed time: 1.02s

d=5: 10635 supports to check
Processed 100 items. Elapsed time: 0.11s. Estimate: 0.19min
Processed all items. Elapsed time: 11.09s

Found 0 potential fundamental models


### $d=6$

In [278]:
n = 6
d = 6
fundamental_models = find_fundamental_models(n, d)
print(f"Found {len(fundamental_models)} potential fundamental models")


d=6: 52284 supports to check
Processed 100 items. Elapsed time: 0.22s. Estimate: 1.9min
Processed all items. Elapsed time: 98.06s
Found 50342 potential fundamental models


In [279]:
filtered = filter_fundamental_models(fundamental_models)
print(f"After filtering non-positive solutions, {len(filtered)} fundamental models were found.")
symmetric = [support for support, w in filtered if support != apply_symmetry(support)]
print(f"After considering symmetric solutions, additional {len(symmetric)} fundamental models were found.")
print(f"We found {len(filtered) + len(symmetric)} fundamental models.")

After filtering non-positive solutions, 3370 fundamental models were found.
After considering symmetric solutions, additional 3340 fundamental models were found.
We found 6710 fundamental models.


In [268]:
save_fundamental_models(fundamental_models, f"section8_n{n}_d{d}.json")

JSON dump has been written to section8_n6_d6.json


### $d=7$

In [258]:
n = 6
d = 7
fundamental_models = find_fundamental_models(n, d)
print(f"Found {len(fundamental_models)} potential fundamental models")


d=7: 180527 supports to check
Processed 100 items. Elapsed time: 0.3s. Estimate: 9.13min
Processed all items. Elapsed time: 468.66s
Found 165719 potential fundamental models


In [259]:
filtered = filter_fundamental_models(fundamental_models)
print(f"After filtering non-positive solutions, {len(filtered)} fundamental models were found.")
symmetric = [support for support, w in filtered if support != apply_symmetry(support)]
print(f"After considering symmetric solutions, additional {len(symmetric)} fundamental models were found.")
print(f"We found {len(filtered) + len(symmetric)} fundamental models.")

After filtering non-positive solutions, 1250 fundamental models were found.
After considering symmetric solutions, additional 1171 fundamental models were found.
We found 2421 fundamental models.


In [264]:
save_fundamental_models(fundamental_models, f"section8_n{n}_d{d}.json")

JSON dump has been written to section8_n6_d7.json


### $d=8$

In [271]:
n = 6
d = 8
fundamental_models = find_fundamental_models(n, d)
print(f"Found {len(fundamental_models)} potential fundamental models")


d=8: 491601 supports to check
Processed 100 items. Elapsed time: 0.31s. Estimate: 25.44min
Processed all items. Elapsed time: 1353.29s
Found 472294 potential fundamental models


In [272]:
filtered = filter_fundamental_models(fundamental_models)
print(f"After filtering non-positive solutions, {len(filtered)} fundamental models were found.")
symmetric = [support for support, w in filtered if support != apply_symmetry(support)]
print(f"After considering symmetric solutions, additional {len(symmetric)} fundamental models were found.")
print(f"We found {len(filtered) + len(symmetric)} fundamental models.")

After filtering non-positive solutions, 324 fundamental models were found.
After considering symmetric solutions, additional 319 fundamental models were found.
We found 643 fundamental models.


In [273]:
save_fundamental_models(fundamental_models, f"section8_n{n}_d{d}.json")

JSON dump has been written to section8_n6_d8.json


### $d = 9$

In [280]:
n = 6
d = 9
fundamental_models = find_fundamental_models(n, d)
print(f"Found {len(fundamental_models)} potential fundamental models")


d=9: 1160287 supports to check
Processed 100 items. Elapsed time: 0.28s. Estimate: 53.59min
Processed all items. Elapsed time: 2633.9s
Found 1159993 potential fundamental models


In [281]:
filtered = filter_fundamental_models(fundamental_models)
print(f"After filtering non-positive solutions, {len(filtered)} fundamental models were found.")
symmetric = [support for support, w in filtered if support != apply_symmetry(support)]
print(f"After considering symmetric solutions, additional {len(symmetric)} fundamental models were found.")
print(f"We found {len(filtered) + len(symmetric)} fundamental models.")

After filtering non-positive solutions, 106 fundamental models were found.
After considering symmetric solutions, additional 92 fundamental models were found.
We found 198 fundamental models.


In [282]:
save_fundamental_models(fundamental_models, f"section8_n{n}_d{d}.json")

JSON dump has been written to section8_n6_d9.json


### $d = 10$

In [283]:
n = 6
d = 10
fundamental_models = find_fundamental_models(n, d)
print(f"Found {len(fundamental_models)} potential fundamental models")


d=10: 2427353 supports to check
Processed 100 items. Elapsed time: 0.35s. Estimate: 140.11min
Processed all items. Elapsed time: 6058.31s
Found 2427088 potential fundamental models


In [284]:
filtered = filter_fundamental_models(fundamental_models)
print(f"After filtering non-positive solutions, {len(filtered)} fundamental models were found.")
symmetric = [support for support, w in filtered if support != apply_symmetry(support)]
print(f"After considering symmetric solutions, additional {len(symmetric)} fundamental models were found.")
print(f"We found {len(filtered) + len(symmetric)} fundamental models.")

After filtering non-positive solutions, 16 fundamental models were found.
After considering symmetric solutions, additional 16 fundamental models were found.
We found 32 fundamental models.


In [285]:
save_fundamental_models(fundamental_models, f"section8_n{n}_d{d}.json")

JSON dump has been written to section8_n6_d10.json


### $d = 11$

In [287]:
n = 6
d = 11
fundamental_models = find_fundamental_models(n, d)
print(f"Found {len(fundamental_models)} potential fundamental models")


d=11: 4710754 supports to check
Processed 100 items. Elapsed time: 0.39s. Estimate: 304.16min
Processed all items. Elapsed time: 13341.55s
Found 4710489 potential fundamental models


In [288]:
filtered = filter_fundamental_models(fundamental_models)
print(f"After filtering non-positive solutions, {len(filtered)} fundamental models were found.")
symmetric = [support for support, w in filtered if support != apply_symmetry(support)]
print(f"After considering symmetric solutions, additional {len(symmetric)} fundamental models were found.")
print(f"We found {len(filtered) + len(symmetric)} fundamental models.")

After filtering non-positive solutions, 2 fundamental models were found.
After considering symmetric solutions, additional 2 fundamental models were found.
We found 4 fundamental models.


In [289]:
save_fundamental_models(fundamental_models, f"section8_n{n}_d{d}.json")

JSON dump has been written to section8_n6_d11.json
